In [ ]:
import os
import pandas as pd
import requests
import time

In [ ]:
path = 'C:\Users\msmu\Desktop\repos\CMU_10718\data\copperunion\anime.csv'

animes = pd.read_csv(path)
ids = list(animes['anime_id'])
print(len(ids))
print(type(ids[0]))

In [ ]:
synopses = {}
# Rate limit information here:
# https://myanimelist.net/forum/?topicid=2142532
# Basically, rate limits approx 300 every 6 minutes or so.

In [ ]:
i = 0
num_fails = 0
fail_ids = []
while i < len(ids):
    id = ids[i]
    url = f'https://api.myanimelist.net/v2/anime/{id}'
    headers = {"X-MAL-CLIENT-ID": "########"}

    # ?fields=id,title,main_picture,alternative_titles,start_date,end_date,
    # synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,
    # created_at,updated_at,media_type,status,genres,my_list_status,num_episodes,
    # start_season,broadcast,source,average_episode_duration,rating,pictures,
    # background,related_anime,related_manga,recommendations,studios,statistics'

    params = {
        'fields': 'main_picture,alternative_titles,synopsis'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        json_data = response.json()
        synopses.update({id: json_data['synopsis']})
        i += 1
        num_fails = 0

    except requests.exceptions.RequestException as e:
        print(f"An error occurred at {i} for id {id}: {e}")
        num_fails += 1
        time.sleep(300)
        if num_fails >= 2:
            i += 1
            fail_ids.append(id)
            num_fails = 0

An error occurred: HTTPSConnectionPool(host='api.myanimelist.net', port=443): Max retries exceeded with url: /v2/anime/30230?fields=rank,mean,alternative_titles (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001D8D368FB50>, 'Connection to proxy.sandia.gov timed out. (connect timeout=None)')))


In [ ]:
print(len(synopses))
print(len(fail_ids))
print(fail_ids)

In [ ]:
df_synopsis = pd.DataFrame(list(synopses.items()), columns=['anime_id', 'synopsis'])

# Convert anime_id to match the data type in df_anime, if necessary
df_synopsis['anime_id'] = df_synopsis['anime_id'].astype(int)

# Merge the two DataFrames on anime_id
df_merged = pd.merge(animes, df_synopsis, on="anime_id", how="left")
df_merged.to_csv("anime_with_synopsis.csv")

# Generate Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
# distiluse-base-multilingual-cased-v1
# distiluse-base-multilingual-cased-v2
# paraphrase-multilingual-MiniLM-L12-v2
# paraphrase-multilingual-mpnet-base-v2

# Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']

# Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)